In [11]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import torch.nn.functional as F
%matplotlib inline
import random
import torchvision
import torchvision.transforms as transforms
import numpy as np
from torchvision import models
batchsize = 256
rowsize = 28
colsize = 28
inchannels = 3
kernelsize = 5
stride = 1
padding = 0
poolsize = 3
fc_size1 = 384
fc_size2 = 192
no_classes = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device available now:', device)

image_transforms = transforms.Compose([
    transforms.Resize(299),
    transforms.CenterCrop(299),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
train = torchvision.datasets.CIFAR10(root='./', train=True, download=True, transform=image_transforms)
evens = list(range(0, len(train), 50))

odds = list(range(1, len(train), 20))
trainset_1 = torch.utils.data.Subset(train, evens)
trainset_2 = torch.utils.data.Subset(train, odds)

test = torchvision.datasets.CIFAR10(root='./', train=False, transform=image_transforms)

traindata = torch.utils.data.DataLoader(trainset_1,batch_size=20, shuffle=True, num_workers=2,pin_memory=True)

testdata = torch.utils.data.DataLoader(trainset_2,batch_size=5, shuffle=False, num_workers=2,pin_memory=True)

transform_inception = transforms.Resize((299,299))


model = models.inception_v3(pretrained=True)
model.aux_logits = False
if torch.cuda.is_available():
    model.cuda()
optimizer = torch.optim.SGD(model.parameters(),lr=0.01, weight_decay = 0.005, momentum = 0.9)
criterion = nn.CrossEntropyLoss().to(device)
def train_test(epochs):
 list_acc = 0
 for i in range(epochs):
  Loss = 0
  num_correct=0
  tot = 0
  model.train()
  for i,data in enumerate(traindata):
    # iterating through every batch
    x_data = data[0].to(device)
    labels = data[1].to(device)
    optimizer.zero_grad()
    outputs = model(x_data)
    # pred[i] = outputs.argmax(dim=1)
    # act[i] = labels.argmax(dim=1)
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
    loss = criterion(outputs, labels)
    loss.backward()
    optimizer.step()
    Loss += loss
    #print(loss)
  list_acc += num_correct/tot
  #print(list_acc)

 num_correct = 0
 tot = 0
 for i,data in enumerate(testdata):
    x_data = data[0].to(device)
    labels = data[1].to(device)
    outputs = model(x_data)
   
    _, pred = torch.max(outputs.data, 1)
    num_correct += (pred == labels).sum().item()
    tot += labels.size(0)
 return (num_correct)/tot , list_acc/epochs

Device available now: cuda
Files already downloaded and verified


In [13]:

list_x = [1,2,3,4,5,6,7,8,9,10]
list_test = []
list_train = []
for i in list_x:
  a,b = train_test(i)
  print(a,b)
  list_test.append(a)
  list_train.append(b)



0.322 0.348
0.49 0.571
0.5092 0.7926666666666667


KeyboardInterrupt: ignored